In [4]:
import fastf1
import pandas as pd
import os
import logging
import warnings

# enable data caching to speed up data loading
logging.getLogger('fastf1').setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore", category=FutureWarning)
cache_folder = "cache_folder"
if not os.path.exists(cache_folder):
    os.makedirs(cache_folder)

fastf1.Cache.enable_cache(cache_folder)

# create a feature for the track type, if some teams perform better here
track_types = {"Melbourne": "Street", "Shanghai": "Permanent" , "Suzuka": "Permanent", "Sakhir": "Permanent", "Jeddah": "Street", "Miami": "Street", "Imola": "Permanent",
                "Monaco": "Street", "Barcelona": "Permanent", "Montréal": "Street", "Spielberg": "Permanent", "Silverstone": "Permanent", "Spa-Francorchamps": "Permanent",
                "Budapest": "Permanent", "Zandvoort": "Permanent", "Monza": "Permanent", "Baku": "Street", "Marina Bay": "Street", "Austin": "Permanent", 
                "Mexico City": "Permanent", "São Paulo": "Permanent", "Las Vegas": "Street", "Lusail": "Permanent", "Yas Island": "Permanent", "Le Castellet": "Permanent"}

# to make sure the team names are the same across our span form 2022-2025
TeamId_map = {"red_bull": "red_bull",
              "mercedes": "mercedes",
              "ferrari": "ferrari",
               "mclaren": "mclaren",
               "alpine": "alpine",
               "aston_martin": "aston_martin",
               "williams": "williams",
               "haas": "haas",
               "alphatauri": "rb", "rb": "rb",
               "alfa": "sauber", "sauber": "sauber", 
               }

# Pirellis classification of the tracks for 2024 season
track_characteristics = {"Sakhir": {"Traction": 4, "Asphalt Grip": 4, "Asphalt Abrasion": 5,
                                    "Track Evolution": 4, "Tyre Stress": 3, "Braking": 4,
                                    "Lateral": 3, "Downforce": 3},
                        "Jeddah": {"Traction": 2, "Asphalt Grip": 4, "Asphalt Abrasion": 2,
                                    "Track Evolution": 4, "Tyre Stress": 3, "Braking": 2,
                                    "Lateral": 3, "Downforce": 2},
                        "Melbourne": {"Traction": 2, "Asphalt Grip": 3, "Asphalt Abrasion": 2,
                                    "Track Evolution": 4, "Tyre Stress": 3, "Braking": 2,
                                    "Lateral": 3, "Downforce": 3},
                        "Suzuka": {"Traction": 3, "Asphalt Grip": 3, "Asphalt Abrasion": 4,
                                    "Track Evolution": 3, "Tyre Stress": 5, "Braking": 2,
                                    "Lateral": 5, "Downforce": 4},
                        "Shanghai": {"Traction": 3, "Asphalt Grip": 2, "Asphalt Abrasion": 4,
                                    "Track Evolution": 5, "Tyre Stress": 4, "Braking": 4,
                                    "Lateral": 4, "Downforce": 3},
                        "Miami": {"Traction": 3, "Asphalt Grip": 3, "Asphalt Abrasion": 2,
                                    "Track Evolution": 5, "Tyre Stress": 3, "Braking": 3,
                                    "Lateral": 3, "Downforce": 2},
                        "Imola": {"Traction": 2, "Asphalt Grip": 2, "Asphalt Abrasion": 3,
                                    "Track Evolution": 2, "Tyre Stress": 3, "Braking": 3,
                                    "Lateral": 3, "Downforce": 3},
                        "Monaco": {"Traction": 5, "Asphalt Grip": 1, "Asphalt Abrasion": 1,
                                    "Track Evolution": 5, "Tyre Stress": 1, "Braking": 2,
                                    "Lateral": 1, "Downforce": 5},
                        "Montréal": {"Traction": 5, "Asphalt Grip": 1, "Asphalt Abrasion": 1,
                                    "Track Evolution": 5, "Tyre Stress": 3, "Braking": 5,
                                    "Lateral": 1, "Downforce": 1},
                        "Barcelona": {"Traction": 3, "Asphalt Grip": 3, "Asphalt Abrasion": 4,
                                    "Track Evolution": 3, "Tyre Stress": 5, "Braking": 3,
                                    "Lateral": 5, "Downforce": 4}, 
                        "Spielberg": {"Traction": 2, "Asphalt Grip": 3, "Asphalt Abrasion": 4,
                                    "Track Evolution": 3, "Tyre Stress": 3, "Braking": 3,
                                    "Lateral": 3, "Downforce": 3},
                        "Silverstone": {"Traction": 3, "Asphalt Grip": 4, "Asphalt Abrasion": 2,
                                    "Track Evolution": 2, "Tyre Stress": 5, "Braking": 2,
                                    "Lateral": 5, "Downforce": 4},
                        "Budapest": {"Traction": 4, "Asphalt Grip": 2, "Asphalt Abrasion": 2,
                                    "Track Evolution": 4, "Tyre Stress": 3, "Braking": 3,
                                    "Lateral": 3, "Downforce": 5},
                        "Spa-Francorchamps": {"Traction": 4, "Asphalt Grip": 4, "Asphalt Abrasion": 4,
                                    "Track Evolution": 3, "Tyre Stress": 5, "Braking": 4,
                                    "Lateral": 5, "Downforce": 2},
                        "Zandvoort": {"Traction": 4, "Asphalt Grip": 3, "Asphalt Abrasion": 3,
                                    "Track Evolution": 4, "Tyre Stress": 5, "Braking": 3,
                                    "Lateral": 4, "Downforce": 4},
                        "Monza": {"Traction": 3, "Asphalt Grip": 4, "Asphalt Abrasion": 2,
                                    "Track Evolution": 4, "Tyre Stress": 3, "Braking": 4,
                                    "Lateral": 2, "Downforce": 1},
                        "Baku": {"Traction": 5, "Asphalt Grip": 2, "Asphalt Abrasion": 1,
                                    "Track Evolution": 5, "Tyre Stress": 3, "Braking": 4,
                                    "Lateral": 1, "Downforce": 2},
                        "Marina Bay": {"Traction": 4, "Asphalt Grip": 3, "Asphalt Abrasion": 3,
                                    "Track Evolution": 5, "Tyre Stress": 2, "Braking": 5,
                                    "Lateral": 2, "Downforce": 5},
                        "Austin": {"Traction": 4, "Asphalt Grip": 3, "Asphalt Abrasion": 3,
                                    "Track Evolution": 4, "Tyre Stress": 4, "Braking": 3,
                                    "Lateral": 4, "Downforce": 4},
                        "Mexico City": {"Traction": 3, "Asphalt Grip": 2, "Asphalt Abrasion": 2,
                                    "Track Evolution": 5, "Tyre Stress": 2, "Braking": 3,
                                    "Lateral": 2, "Downforce": 5},
                        "São Paulo": {"Traction": 3, "Asphalt Grip": 3, "Asphalt Abrasion": 2,
                                    "Track Evolution": 5, "Tyre Stress": 3, "Braking": 3,
                                    "Lateral": 3, "Downforce": 4},
                        "Las Vegas": {"Traction": 2, "Asphalt Grip": 1, "Asphalt Abrasion": 2,
                                    "Track Evolution": 5, "Tyre Stress": 4, "Braking": 3,
                                    "Lateral": 2, "Downforce": 1},
                        "Lusail": {"Traction": 3, "Asphalt Grip": 3, "Asphalt Abrasion": 2,
                                    "Track Evolution": 4, "Tyre Stress": 5, "Braking": 3,
                                    "Lateral": 5, "Downforce": 4},
                        "Yas Island": {"Traction": 4, "Asphalt Grip": 3, "Asphalt Abrasion": 3,
                                    "Track Evolution": 4, "Tyre Stress": 3, "Braking": 4,
                                    "Lateral": 3, "Downforce": 3},
                        "Le Castellet": {"Traction": 4, "Asphalt Grip": 3, "Asphalt Abrasion": 3,
                                    "Track Evolution": 3, "Tyre Stress": 4, "Braking": 2,
                                    "Lateral": 4, "Downforce": 3} # 2022 season
                        
                        }

In [5]:
import numpy as np
rounds = np.arange(1, 12) # Austrian GP

all_results = []
for rnd in rounds:
    
    session = fastf1.get_session(2025, rnd, "r")
    session.load()  

    results = session.results[["Abbreviation", "Points"]].copy()
    
    results["round"] = rnd 
    
    all_results.append(results)


season_results_2025 = pd.concat(all_results, ignore_index=True)


In [6]:
from sklearn.preprocessing import StandardScaler

all_results = []
for rnd in rounds:
    

    race_session = fastf1.get_session(2025, rnd, "r")
    race_session.load()


    data = race_session.results[["Abbreviation", "ClassifiedPosition", "GridPosition", "TeamId"]].copy()
    
    location = race_session.event.get("Location", None)
    data["Track_location"] = location
    data["Round"] = rnd
    data["Season"] = 2025

    data["Track_characteristics"] = data["Track_location"].map(track_characteristics)

    track_features = data["Track_characteristics"].apply(pd.Series)

    data = data.join(track_features)
    data.drop(columns=("Track_characteristics"), inplace=True)


    try:
        fp2_session = fastf1.get_session(2025, rnd, "fp2")
        fp2_session.load()

        all_laps = fp2_session.laps
        all_laps = all_laps[all_laps["IsAccurate"]]

        race_sim_dict = {}
        for driver in data["Abbreviation"]:
            

            laps = all_laps[all_laps["Driver"] == driver]
            fast_s1 = laps["Sector1Time"].min()
            fast_s2 = laps["Sector2Time"].min()
            fast_s3 = laps["Sector3Time"].min() 

            delta = pd.Timedelta(seconds=0.3)

            clean_sector_1 = laps[laps["Sector1Time"] <= fast_s1 + delta]
            avg_s1 = clean_sector_1["Sector1Time"].mean()

            # Sector 2
            clean_sector_2 = laps[laps["Sector2Time"] <= fast_s2 + delta]
            avg_s2 = clean_sector_2["Sector2Time"].mean()

            # Sector 3
            clean_sector_3 = laps[laps["Sector3Time"] <= fast_s3 + delta]
            avg_s3 = clean_sector_3["Sector3Time"].mean()



            avg_laptime = (avg_s1 + avg_s2 + avg_s3).total_seconds()
            race_sim_dict[driver] = avg_laptime    

        data["race_sim"] = data["Abbreviation"].map(race_sim_dict)
        scaler = StandardScaler()
        data["race_sim"] = scaler.fit_transform(data["race_sim"].values.reshape(-1, 1))
    except:
        fp2_session = fastf1.get_session(2025, rnd, "s")
        fp2_session.load()

        all_laps = fp2_session.laps
        all_laps = all_laps[all_laps["IsAccurate"]]

        race_sim_dict = {}
        for driver in data["Abbreviation"]:
            

            laps = all_laps[all_laps["Driver"] == driver]
            fast_s1 = laps["Sector1Time"].min()
            fast_s2 = laps["Sector2Time"].min()
            fast_s3 = laps["Sector3Time"].min() 

            delta = pd.Timedelta(seconds=0.5)

            clean_sector_1 = laps[laps["Sector1Time"] <= fast_s1 + delta]
            avg_s1 = clean_sector_1["Sector1Time"].mean()

            # Sector 2
            clean_sector_2 = laps[laps["Sector2Time"] <= fast_s2 + delta]
            avg_s2 = clean_sector_2["Sector2Time"].mean()

            # Sector 3
            clean_sector_3 = laps[laps["Sector3Time"] <= fast_s3 + delta]
            avg_s3 = clean_sector_3["Sector3Time"].mean()



            avg_laptime = (avg_s1 + avg_s2 + avg_s3).total_seconds()
            race_sim_dict[driver] = avg_laptime    

        data["race_sim"] = data["Abbreviation"].map(race_sim_dict)
        scaler = StandardScaler()
        data["race_sim"] = scaler.fit_transform(data["race_sim"].values.reshape(-1, 1))

    if rnd - 5 >= 1:
        start_idx = rnd - 5
    else:
        start_idx = 1
    end_idx = rnd - 1 

    wanted = season_results_2025[(season_results_2025["round"] >= start_idx) & (season_results_2025["round"] <= end_idx)]

    weights = [0.2, 0.4, 0.6, 0.8, 1]

    for driver in wanted["Abbreviation"].unique():
        driver_result =  wanted[wanted["Abbreviation"] == driver].sort_values("round")

        weighted_form = 0

        for i, (_, row) in enumerate(driver_result.iterrows()):
            if i < len(weights):
                weighted_form += row["Points"] * weights[i]
        data.loc[data["Abbreviation"] == driver, "Form-score"] = weighted_form
    all_results.append(data)

season_results_2025 = pd.concat(all_results, ignore_index=True)

season_results_2025["Form-score"] = season_results_2025["Form-score"].fillna(0) #fix first race and other errors that could exist

season_results_2025["Track_type"] = season_results_2025["Track_location"].map(track_types)
season_results_2025["TeamId"] = season_results_2025["TeamId"].map(TeamId_map)


In [8]:
season_results_2025.to_parquet("2025_data.parquet", index=False)